<a href="https://colab.research.google.com/github/alessandronicolini/IncrementalLearning/blob/main/mnemonics_ready_to_test_with_new_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install 'import_ipynb'
!pip3 install 'tqdm'
 
!rm -r IncrementalLearning
# upload work files from your git hub repository
import sys
 
!git clone https://github.com/alessandronicolini/IncrementalLearning.git # clone proj repository
!rm -rf IncrementalLearning/README.md 
!rm -rf IncrementalLearning/baselines.ipynb
 
path = 'IncrementalLearning/'
if path not in sys.path:
    sys.path.append('IncrementalLearning/')
 
!pip3 install import_ipynb

Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 577 (delta 65), reused 0 (delta 0), pack-reused 474
Receiving objects: 100% (577/577), 652.60 KiB | 12.08 MiB/s, done.
Resolving deltas: 100% (340/340), done.


In [2]:
import numpy as np
import torch
from torchvision import transforms
from torchvision.datasets import VisionDataset
from PIL import Image
import random
import torchvision
ROOT = './data'
class ilCIFAR100(VisionDataset):
    """
    Extends CIFAR100 class. Split the dataset into 10 batches, each one containing 10 classes.
    You can retrieve the batches from the attribute "batches", it has different structure according to
    test and train CIFAR100 splits:
        - train -> batches is a dictionary {0:{'train':indexes, 'val':indexes}...} 
        - test -> batches is a dictionary {0:indexes...}
    where the keys are the batch number.

    Args:
        root (string): Root directory of dataset where directory
            `cifar-10-batches-py` exists or will be saved to if download is set to True.
        seed(int): used to ensure reproducibility in shuffling operations.
        val_size(float, optional): between 0 and 1, fraction of data used for validation.
        train (bool, optional): If True, creates dataset from training set, otherwise
            creates from test set.
        transform (callable, optional): A function/transform that takes in an PIL image
            and returns a transformed version. E.g, `transforms.RandomCrop`
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
    """
    def __init__(self,classes_per_batch, seed, val_size=0.1, train=True, transform=None, target_transform=False, 
    download=True):
        
        super(ilCIFAR100, self).__init__(root=0)
        self.classes_per_batch=classes_per_batch

        
        self.__rs = seed # set random seed 
        self.train=train
        self.__transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
        ])

        self.__transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
        ])
        # if train:
        #     self.batches = self.__make_train_batches(val_size)
        # else:
        #     self.batches = self.__make_test_batches()
        if self.train == 'train':
          self.dataset = torchvision.datasets.CIFAR100(root=ROOT, train=True,
                                            download=True, transform=self.__transform_train)
        elif self.train == 'exemplars':
          self.dataset = torchvision.datasets.CIFAR100(root=ROOT, train=True,
                                            download=True, transform=self.__transform_test)
        else:
          self.dataset = torchvision.datasets.CIFAR100(root=ROOT, train=False,
                                        download=True, transform=self.__transform_test)
          
        self.targets = np.array(self.dataset.targets) # make targets an array to exploit masking
        random.seed(seed)
        self.classes = random.sample(range(0, 100), 100)
        #self.classes = self.classes.reshape((10, -1)) # each row contains the classes for the corrisponding batch
        #print(self.classes)
        self.__dictionary = {}
        for i, c in enumerate(self.classes):
          self.__dictionary[c] = i


    def get_dict(self):
      return self.__dictionary
    def __getitem__(self, index):
        return index,self.dataset.__getitem__(index)[0],self.dataset.__getitem__(index)[1]
    def __len__(self):
        return self.dataset.__len__()
    def getbatches(self):
      classlist=self.classes
      batches=[]
      for i in range(0,int(100/self.classes_per_batch)):
        #print(i)
        batch=classlist[int(i*self.classes_per_batch):int(i*10+self.classes_per_batch)]
        batches.append(batch)
      return batches
    def get_batch_indexes(self):
      classlist=self.classes
      numclass=self.classes_per_batch
      batch_indexes=[]
      for i in range(0,int(100/self.classes_per_batch)):
        batch=classlist[int(i*numclass):int(i*numclass+numclass)]
        mask=np.isin(self.targets,batch)
        indexes=np.array(np.arange(len(self.dataset.targets)))
        indexes=indexes[mask]
        batch_indexes.append(indexes)
      return batch_indexes
    def get_class_indexes(self,label):
      indexes = np.array(np.arange(len(self.dataset.targets)))
      labels = self.dataset.targets
      mask = np.isin(labels, label)
      indexes = indexes[mask]

      return indexes
    def get_train_val(self,valid):
      batches=self.get_batch_indexes()
      train=[]
      val=[]
      for batch in batches:
        #print(type(batch))
        random.shuffle(batch)
        valbatch=batch[0:int(valid*len(batch))]
        trainbatch=batch[int(valid*len(batch)):]
        train.append(trainbatch)
        val.append(valbatch)
      return train,val
import torch
import torch.optim as optim
import torchvision
import time
import os
import argparse
import numpy as np

def tensor2im(input_image, imtype=np.uint8):
    mean = [0.5071, 0.4867, 0.4408]
    std = [0.2675, 0.2565, 0.2761]
    if not isinstance(input_image, np.ndarray):
        if isinstance(input_image, torch.Tensor):
            image_tensor = input_image.data
        else:
            return input_image
        image_numpy = image_tensor.cpu().detach().float().numpy()
        if image_numpy.shape[0] == 1:
            image_numpy = np.tile(image_numpy, (3, 1, 1))
        for i in range(len(mean)): 
            image_numpy[i] = image_numpy[i] * std[i] + mean[i]
        image_numpy = image_numpy * 255
        image_numpy = np.transpose(image_numpy, (1, 2, 0))
    else:
        image_numpy = input_image
    return image_numpy.astype(imtype)

In [3]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from PIL import Image
import torchvision
import torchvision.transforms as transforms
import math
from sklearn.preprocessing import normalize
import copy
import torchvision.datasets as dsets
import torchvision.models as models
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from torch.utils.data import Subset, DataLoader, Dataset
import random
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import pandas as pd

import import_ipynb
#from IncrementalLearning.cifar100 import ilCIFAR100

from IncrementalLearning.resnet_cifar import resnet32
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

importing Jupyter notebook from /content/IncrementalLearning/resnet_cifar.ipynb


In [4]:
class mnemonics():
  def __init__(self, randomseed):
    self.device = 'cuda'
    self.model = resnet32(num_classes=100).to(self.device)
    self.feature_extractor = self.model.features
    self.temp_model = None
    self.lr = 2
    self.gamma = 0.2
    self.weight_decay = 1e-5 
    self.milestones = [49,63]
    self.batch_size = 128
    self.numepochs = 1
    self.n_classes = 0
    self.n_known = 0
    self.feature_size=64
    self.momentum=0.9
    self.criterion = nn.BCEWithLogitsLoss()

    self.NUM_BATCHES=10
    self.randomseed=randomseed
    self.trainloader=None
    self.testloader=None
    self.CLASSES_PER_BATCH=10

    self.original_training_set = ilCIFAR100(self.CLASSES_PER_BATCH, self.randomseed, train = 'train')
    self.original_exemplar_set = ilCIFAR100(self.CLASSES_PER_BATCH, self.randomseed, train = 'exemplars')
    self.original_test_set = ilCIFAR100(self.CLASSES_PER_BATCH,self.randomseed, train= 'test')

    self.last_test = None
    self.y_pred = []
    self.y_test = []

    self.cumulative_class_mean = []

    self.classes_seen=0
    self.diz = self.original_training_set.get_dict()

    self.exemplar_features_mean = None
    # lista di liste, ogni lista contiene gli exemplars di una classe
    self.exemplar_sets_idxs = [] # mn_exemplat_sets
    # lista unica, tutti gli indici degli exemplar
    self.exemplar_idxs = []


  def update_params(self, 
                    m,
                    finetuning_idxs, 
                    training_idxs, 
                    mnemonics_to_optimize, 
                    batch_size,
                    new=True,
                    lr=0.01, 
                    momentum=0.9, 
                    weight_decay=1e-5, 
                    milestones=[10, 20, 30, 40],
                    gamma=0.5, 
                    tuning_epochs=2,
                    updating_epochs=30):
    
    """
    finetuning_idxs = indexes of current task elements
    mnemonics_idxs = indexes of exemplar elements
    mnemonics_to_optimize = the optimized parameters in the update phase
    """

    # make a copy of the model
    model_copy = copy.deepcopy(self.model)
    model_copy.train()
    model_copy.to(self.device)

    # define the loss
    # criterion = nn.BCEWithLogitsLoss()

    # FINE TUNING FOR 1 EPOCH eq. 8 --------------------------------------------
    
    # define optimizer and scheduler for fine tuning phase
    optimizer = optim.SGD(model_copy.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    
    # create the subset dataset to load the data you want, and the loader
    finetuning_labels = np.array([self.original_training_set.__getitem__(idx)[2] for idx in finetuning_idxs], dtype=int)
    meta_idxs = [i for i in range(len(finetuning_idxs))]
    random.shuffle(meta_idxs)

    # split the meta idxs in batches
    n_batches = int(np.floor(len(finetuning_idxs)/batch_size))
    meta_idxs_batches = []
    for i in range(n_batches):
      meta_idxs_batches.append(np.array(meta_idxs[batch_size*i:batch_size*(i+1)]))
    meta_idxs_batches.append(np.array(meta_idxs[batch_size*n_batches:]))

    # now fine tune the copied model
    for epoch in range(tuning_epochs):
      for meta_idxs_batch in meta_idxs_batches:
        inputs = mnemonics_to_optimize[0][meta_idxs_batch] # are already in cuda
        labels = finetuning_labels[meta_idxs_batch]
        labels = torch.tensor([self.diz[c] for c in labels])
        labels = labels.to(self.device)
        optimizer.zero_grad()
        outputs = model_copy(inputs)
        labels_encoded = F.one_hot(labels,100).float().to(self.device)
        loss = self.criterion(outputs, labels_encoded)
        loss.backward()
        optimizer.step()
    

    # UPDATE THE MNEMONICS eq.9/10 ---------------------------------------------
    
    model_copy.eval()
    
    optimizer = optim.SGD(mnemonics_to_optimize, lr=lr, momentum=momentum, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)
    

    if new:
      exlvl_training = Subset(self.original_training_set, training_idxs)
      exlvl_loader = DataLoader(exlvl_training, batch_size=batch_size, shuffle=True, num_workers=4)

      current_task_labels = set([self.original_training_set.__getitem__(idx)[2] for idx in training_idxs])
      new_dict = {label:new_label for label, new_label in zip(current_task_labels, range(10))}

      new_class_mean = {new_dict[key] : value for key, value in self.cumulative_class_mean.items()}
      #print('newdict', new_dict)
      #print('cumulative', self.cumulative_class_mean)
      #print('new class mean', new_class_mean)
      #print('sorted', sorted(new_class_mean.items()))
      means_ready = torch.Tensor(list(new_class_mean.values()))

    
    else:
      exlvl_training = Subset(self.original_exemplar_set, training_idxs)
      exlvl_loader = DataLoader(exlvl_training, batch_size=batch_size, shuffle=True, num_workers=4)


    for epoch in tqdm(range(updating_epochs)):
      for _, inputs, labels in exlvl_loader:

        if new:
          labels = torch.tensor([new_dict[c.item()] for c in labels])
        else:
          labels = torch.tensor([self.diz[c.item()] for c in labels])

        labels = labels.to(self.device)
        inputs = inputs.to(self.device)
        out_features = model_copy.features(inputs)
        # compute features mean of mnemonics for each class
        all_class_means = torch.zeros((0, 64))
        all_class_means = all_class_means.to(self.device)
        n_classes = int(len(finetuning_labels)/m)
        if new:
          the_logits = F.linear(F.normalize(out_features, p=2, dim=1), F.normalize(means_ready, p=2, dim=1))
        else:
          for i in range(n_classes): # how many classes
            mnemonics_features = model_copy.features(mnemonics_to_optimize[0][i*m:(i+1)*m])
            this_class_means = torch.mean(mnemonics_features, dim=0) # size 64
            this_class_means = torch.unsqueeze(this_class_means, dim=0) # add the second dimension
            all_class_means = torch.cat((all_class_means, this_class_means), dim=0)
          the_logits = F.linear(F.normalize(out_features, p=2, dim=1), F.normalize(all_class_means, p=2, dim=1))
        labels_encoded = F.one_hot(labels,100).float().cuda()
        loss = F.cross_entropy(the_logits, labels) # al secondo batch di classi per i new mnemonics le uscite sono sempre 10 ma le label vanno da 10 a 19
        loss.backward()
        optimizer.step()

  def exemplar_level_optimization(self, m, task_num, current_task_indices):  
    
    # UPDATING NEW EXEMPLAR-----------------------------------------------------

    # isola gli indici dei nuovi exemplars
    new_exemplar_idxs = []
    for idxs in self.exemplar_sets_idxs[-10:]:
      new_exemplar_idxs += idxs

    # ora ottieni gli mnemonics che poi sono da ottimizzare
    new_mnemonics_data = torch.zeros((10*m, 3, 32, 32))

    for i, idx in enumerate(new_exemplar_idxs):
      new_mnemonics_data[i, :, :, :] = self.original_training_set.__getitem__(int(idx))[1]

    new_mnemonics = nn.ParameterList()
    new_mnemonics.append(nn.Parameter(new_mnemonics_data))
    new_mnemonics.to(self.device)
    
    #print(new_mnemonics[0][0])

    options_new ={'finetuning_idxs': new_exemplar_idxs, 
                  'training_idxs': current_task_indices, 
                  'mnemonics_to_optimize':  new_mnemonics,  
                  'batch_size':128,
                  'm':m}

    print('---start mnemonics updating---')

    self.update_params(**options_new)    

    for i, idx in enumerate(new_exemplar_idxs):
      self.original_training_set.dataset.data[idx] = tensor2im(new_mnemonics[0][i])

    
    # UPDATING OLD EXEMPLARS ---------------------------------------------------
    '''
    if task_num:
      # decidi quanti elementi ha ogni exemlar set in a e in b a seconda se m è 
      # pari o dispari
      if m%2:
        l_a = int((m+1)/2)
      else:
        l_a = int(m/2)
      l_b = int(m-l_a)

      # isola gli indici dei vecchi exemplars, dividendoli in due parti
      # ogni classe deve avere circa la metà degli exemplar originali
      old_exemplar_idxs_a = []
      old_exemplar_idxs_b = []
      
      for idxs in self.exemplar_sets_idxs[:-10]:
        old_exemplar_idxs_a += idxs[:l_a]
        old_exemplar_idxs_b += idxs[l_a:]

      old_mnemonics_data_a = torch.zeros((task_num*10*l_a, 3, 32, 32))
      old_mnemonics_data_b = torch.zeros((task_num*10*l_b, 3, 32, 32))

      for i, idx in enumerate(old_exemplar_idxs_a):
        old_mnemonics_data_a[i, :, :, :] = self.original_training_set.__getitem__(int(idx))[1]
      
      for i, idx in enumerate(old_exemplar_idxs_b):
          old_mnemonics_data_b[i, :, :, :] = self.original_training_set.__getitem__(int(idx))[1]
      
      old_mnemonics_a = nn.ParameterList()
      old_mnemonics_a.append(nn.Parameter(old_mnemonics_data_a))
      old_mnemonics_a.to(self.device)
      old_mnemonics_b = nn.ParameterList()
      old_mnemonics_b.append(nn.Parameter(old_mnemonics_data_b))
      old_mnemonics_b.to(self.device)

      options_old_a = {'finetuning_idxs':old_exemplar_idxs_a, 
                       'training_idxs':old_exemplar_idxs_b, 
                       'mnemonics_to_optimize':old_mnemonics_a, 
                       'batch_size':128,
                       'm': l_a,
                       'new':False}

      options_old_b = {'finetuning_idxs':old_exemplar_idxs_b, 
                       'training_idxs':old_exemplar_idxs_a, 
                       'mnemonics_to_optimize':old_mnemonics_b, 
                       'batch_size':128,
                       'm':l_b,
                       'new':False}

      self.update_params(**options_old_a) 
      self.update_params(**options_old_b)

      # CONVERT AND STORE UPDATED EXEMPLAR as numpy array

      for i, idx in enumerate(old_exemplar_idxs_a):
        self.original_training_set.dataset.data[idx] = tensor2im(old_mnemonics_a[0][i])

      for i, idx in enumerate(old_exemplar_idxs_b):
        self.original_training_set.dataset.data[idx] = tensor2im(old_mnemonics_b[0][i])
    
    '''

    # FINE TUNE THE CURRENT NET ON ALL THE EXEMPLARS COLLECTED 'TILL NOW
    

  def model_level_optimization(self):
    
    old_model = copy.deepcopy(self.model)
    old_model.eval()
    old_model.to(self.device)
    n_classes = self.classes_seen+self.CLASSES_PER_BATCH
    print(n_classes)
    optimizer = optim.SGD(self.model.parameters(), lr=self.lr, momentum=self.momentum, weight_decay=self.weight_decay)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.milestones, gamma=self.gamma)
    for epoch in tqdm(range(self.numepochs)):
        
      for _, inputs, labels in self.trainloader:
        inputs = inputs.float().to(self.device)
        labels = torch.tensor([self.diz[c.item()] for c in labels])

        labels=labels.to(self.device)
        optimizer.zero_grad()
        outputs=self.model(inputs)

        labels_encoded = F.one_hot(labels,100).float().to(self.device) #CAMBIARE ONE_HOT
        
        if self.classes_seen:
          old_target = old_model(inputs).to(self.device)
          old_target = torch.sigmoid(old_target).to(self.device)
          
          target = torch.cat((old_target[:,:self.classes_seen], labels_encoded[:, self.classes_seen:]), dim=1)
          loss = self.criterion(outputs, target)
        else:
          loss = self.criterion(outputs, labels_encoded) 

        loss.backward()
        optimizer.step()
      
      scheduler.step()


  def classify_nme(self, input_batch):
    min_distances = float('inf')*torch.ones(len(input_batch)).to(self.device) # shape: batch_size --> 128
    y_pred = torch.zeros(len(input_batch), dtype=torch.int8).to(self.device) # shape: batch_size --> 128
    input_features = self.model.features(input_batch) # shape: (batch_size, feature_size) --> (128, 64)

    for i in range(len(self.exemplar_sets_idxs)):
      ex_mean = self.exemplar_means[i,:]

      # compute distances between inputs features and exemplar set means
      pdist = nn.PairwiseDistance(p=2)
      distances = pdist(input_features, ex_mean) # shape: batch_size --> 128

      # update min distancies and predicted labels
      mask = distances < min_distances
      min_distances[mask] = distances[mask]
      y_pred[mask] = self.exemplar_labels[i]

    return y_pred
    

  def get_new_exemplars(self, batch_indexes, m):
    self.exemplar_sets_idxs.append(random.sample(list(batch_indexes), m))


  def reduce_old_exemplars(self, m):
    for i, set_i in enumerate(self.exemplar_sets_idxs):
      self.exemplar_sets_idxs[i] = random.sample(set_i, m)

  def __accuracy_fc(self, dl, mapper):
    total = 0.0
    correct = 0.0
    for  _, images, labels in dl:
      labels = torch.tensor([torch.tensor(mapper[c.item()]) for c in labels])
      labels = labels.to(self.device)
      images = images.to(self.device)
      outputs = self.model(images)
      _, preds = torch.max(outputs, dim=1)
      total += len(labels)
      correct += torch.sum(preds == labels).item()

    acc = correct / total
    return acc


  def __accuracy_nme(self, dl):
    
    total = 0.0
    correct = 0.0
    
    for  _, images, labels in dl:
      labels = labels.to(self.device)
      images = images.to(self.device)
      preds = self.classify_nme(images)
      total += len(labels)
      correct += torch.sum(preds == labels).item()

      if self.last_test:
        self.y_pred += preds.tolist()
        self.y_test += labels.tolist()

    acc = correct / total
    return acc


  def plot_confusion_matrix(self):
 
    cm = confusion_matrix(self.y_test, self.y_pred)
    cm = np.log(cm+1)
    fig, ax = plt.subplots(figsize=(7,7))
    sns.heatmap(cm, square=True, cbar=False, ax=ax, cmap=plt.get_cmap('seismic'))
    ax.set_xticks(np.linspace(19,99,5))
    ax.set_yticks(np.linspace(19,99,5))
    ax.set_xticklabels([20,40,60,80,100], rotation=0)
    ax.set_yticklabels([20,40,60,80,100], rotation=0)
    ax.set_title("iCaRL")
    ax.set_xlabel("Predicted class")
    ax.set_ylabel("True class")
    plt.savefig("iCaRL_"+str(self.randomseed)+"_cm.png")
    plt.show()
    return cm

  def plot_data(self, dl):

    from sklearn.manifold import TSNE

    labels_array = torch.zeros(0).to('cuda')
    dataset_to_reduce = torch.zeros((0, 64)).to('cuda')

    for  _, images, labels in dl:
      labels = labels.to(self.device)
      images = images.to(self.device)
      dataset_to_reduce = torch.cat((dataset_to_reduce, self.feature_extractor(images)))
      labels_array = torch.cat((labels_array, labels))


    print('example:', dataset_to_reduce[0])
    print('len of dataset to reduce:', dataset_to_reduce[0].shape)
    print('example:', dataset_to_reduce[0][0])
    X_transformed = TSNE(n_components=2).fit_transform(dataset_to_reduce.cpu())
    
    print('shape of transofmed', X_transormed.shape)
    
    fig, ax = plt.subplots(figsize=(15,15))
    ax.scatter(X_transformed[:,0], X_transormed[:,1], c = labels_array)
    plt.show()

   # for idx in data_idxs:
    #  dataset_to_reduce.append([dataset.__getitem__(idx)[1], dataset.__getitem__(idx)[2]])
    
  def trainer(self):
    
    train_indices = self.original_training_set.get_batch_indexes()
    test_indices = self.original_test_set.get_batch_indexes()
    batches=self.original_training_set.getbatches()
    current_test_indexes=[]
    test_acc = []
    self.last_test = False

    for i in range(self.NUM_BATCHES):

      if i == self.NUM_BATCHES-1:
        self.last_test = True

      current_exemplar_indices = np.array([], dtype=int)

      for exemplar_set in self.exemplar_sets_idxs:
        current_exemplar_indices = np.concatenate([current_exemplar_indices, np.array(exemplar_set)])

      exemplar_dataset = Subset(self.original_exemplar_set, current_exemplar_indices)

      train_dataset = Subset(self.original_training_set, train_indices[i])
      current_test_indexes += test_indices[i].tolist()
      test_dataset = Subset(self.original_test_set,current_test_indexes)
      self.train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=4, drop_last=True)
      self.testloader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=4, drop_last=True)        
      

      if i == 0:
        self.trainloader = self.train_loader
      else:
        self.trainloader = DataLoader(torch.utils.data.ConcatDataset([train_dataset, exemplar_dataset]), batch_size=self.batch_size, shuffle=True,
          num_workers=4, pin_memory=True)
        
        print('lunghezza train loader:', len(self.trainloader))
        
      self.model.train()
      self.model_level_optimization()    
      self.classes_seen += 10
      self.model.eval() # Set Network to evaluation mode

      #self.plot_data(self.trainloader)

      # update exemplars number
      m=int(2000/(int(i*10+10)))

      # reduce the number of each exemplars set
      self.reduce_old_exemplars(m) 
      
      self.cumulative_class_mean = {}

      for classlabel in batches[i]:
        indexes_class = self.original_training_set.get_class_indexes(classlabel)
        #current_class = Subset(self.original_training_set, indexes_class)
        self.get_new_exemplars(indexes_class, m)
        indexes_class = self.original_training_set.get_class_indexes(classlabel)
        current_class = Subset(self.original_training_set, indexes_class)

        loader = torch.utils.data.DataLoader(current_class, batch_size=self.batch_size,shuffle=False, num_workers=4)
        features = np.zeros((0, 64))
        with torch.no_grad():
          for indexes, images, labels in loader:
            images = images.cuda()
            feature = self.feature_extractor(images).data.cpu().numpy()
            feature = normalize(feature, axis=1, norm='l2')
            features = np.concatenate((features,feature), axis=0)

        class_mean = np.mean(features, axis=0)
        class_mean = class_mean / np.linalg.norm(class_mean)  # Normalize

        self.cumulative_class_mean[classlabel] = class_mean
        

      
      if i != 9:
        self.exemplar_level_optimization(m, i, self.original_training_set.get_batch_indexes()[i])

      # compute means of exemplar set
      # cycle for each exemplar set
      self.exemplar_means = torch.zeros((0, self.feature_size), dtype=torch.float).to(self.device)
      self.exemplar_labels = []
      for i in range(len(self.exemplar_sets_idxs)):
        exemplars_dataset = Subset(self.original_training_set, self.exemplar_sets_idxs[i])
        exemplars_loader = torch.utils.data.DataLoader(exemplars_dataset, batch_size=self.batch_size, shuffle=False, num_workers=4)
        ex_features = torch.zeros((0, self.feature_size), dtype=torch.float).to(self.device) # alla fine shape: (len(exemplar_set), feature_size) --> (m, 64)
      
        with torch.no_grad():
          _, _, exemplar_label = self.original_training_set.__getitem__(self.exemplar_sets_idxs[i][0]) 
          self.exemplar_labels.append(exemplar_label)
          # cycle for each batch in the current exemplar set
          for _,  exemplars, _ in exemplars_loader:
          
            # get exemplars features
            exemplars = exemplars.to(self.device)
            features = self.model.features(exemplars) # shape: (len(exemplars), feature_size)
          
            # normalize 
            feature_norms = torch.norm(features, p=2, dim=1) # shape: len(exemplars)
            feature_norms.unsqueeze_(1) # shape: (len(exemplars), 1)
            features = features/feature_norms
          
            # concatenate over columns
            ex_features = torch.cat((ex_features, features), dim=0)
          
        # compute current exemplar set mean and normalize it
        ex_mean = torch.mean(ex_features, dim=0) # shape: feature_size --> 64
        ex_mean = ex_mean/torch.norm(ex_mean)
        ex_mean.unsqueeze_(0) # shape: (1, feature_size) --> (1, 64)
        self.exemplar_means = torch.cat((self.exemplar_means, ex_mean), dim=0) # shape: (n_examplar sets, feature size)
      

      print('accuracy on training set:', 100*self.__accuracy_fc(self.trainloader,self.diz))
      # print('accuracy on test set:', self.__accuracy_on(self.testloader,self,self.diz))
      current_test_acc = self.__accuracy_nme(self.testloader)
      print('accuracy on test set:', 100*current_test_acc)
      print('-' * 80)
      test_acc.append(current_test_acc)

    # compute comfusion matrix and save results
    cm = self.plot_confusion_matrix()
    with open('iCaRL_'+str(self.randomseed)+"_cm", 'wb') as file:
      pickle.dump(cm, file, protocol=pickle.HIGHEST_PROTOCOL)
    with open('iCaRL_'+str(self.randomseed)+"_testacc", 'wb') as file:
      pickle.dump(test_acc, file, protocol=pickle.HIGHEST_PROTOCOL)


In [5]:
method = mnemonics(randomseed=203)
method.trainer()

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
10


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")


---start mnemonics updating---
newdict {96: 0, 33: 1, 3: 2, 5: 3, 11: 4, 76: 5, 78: 6, 62: 7, 27: 8, 30: 9}
cumulative {11: array([0.11478377, 0.09573872, 0.11334106, 0.07356598, 0.10375725,
       0.07664447, 0.21082932, 0.08269825, 0.03474149, 0.12407912,
       0.07467396, 0.14944355, 0.07321193, 0.26733924, 0.07716078,
       0.10625511, 0.1193089 , 0.07339835, 0.04522939, 0.15713954,
       0.16908594, 0.09109797, 0.1118566 , 0.04654253, 0.07798271,
       0.12124483, 0.10925392, 0.18708963, 0.13651516, 0.07134686,
       0.25613617, 0.12832826, 0.09289955, 0.01988344, 0.23898355,
       0.06482295, 0.04691762, 0.12228998, 0.0579396 , 0.0986282 ,
       0.04682863, 0.18872884, 0.0637376 , 0.17143292, 0.17728179,
       0.10867876, 0.22670674, 0.10237628, 0.06233365, 0.20415271,
       0.1183183 , 0.13471816, 0.05216757, 0.06655449, 0.1713834 ,
       0.07274779, 0.08313703, 0.13598381, 0.07042525, 0.05905767,
       0.14290069, 0.11442528, 0.10125687, 0.08562214]), 5: array([0.137

tensor([33, 30, 76, 33, 96,  3, 27, 96, 11, 27, 96,  5, 78,  3, 76, 78, 33, 62,
         5, 62,  3,  5, 11, 96, 30, 33, 76,  5,  5, 11, 96,  5, 27, 30, 62, 76,
        11,  3, 62, 30, 78, 62, 30, 33, 30,  5,  5, 96, 27, 78, 62, 33, 78, 33,
        62, 27,  5,  3,  3, 76, 27,  3, 96, 78, 78, 62, 30, 96, 30,  5, 96, 96,
        30,  3, 27,  3, 27, 33, 62,  3, 62, 76, 78, 76, 11, 76, 33,  5,  5, 30,
        78,  3, 27, 62, 33, 27, 78,  3,  5, 33, 96,  3, 11,  3, 62,  5,  5, 78,
        30, 96, 30, 30, 11, 30, 62, 11, 62, 33, 76, 76, 27, 33, 33,  3,  5, 76,
        30,  3])
tensor([1, 9, 5, 1, 0, 2, 8, 0, 4, 8, 0, 3, 6, 2, 5, 6, 1, 7, 3, 7, 2, 3, 4, 0,
        9, 1, 5, 3, 3, 4, 0, 3, 8, 9, 7, 5, 4, 2, 7, 9, 6, 7, 9, 1, 9, 3, 3, 0,
        8, 6, 7, 1, 6, 1, 7, 8, 3, 2, 2, 5, 8, 2, 0, 6, 6, 7, 9, 0, 9, 3, 0, 0,
        9, 2, 8, 2, 8, 1, 7, 2, 7, 5, 6, 5, 4, 5, 1, 3, 3, 9, 6, 2, 8, 7, 1, 8,
        6, 2, 3, 1, 0, 2, 4, 2, 7, 3, 3, 6, 9, 0, 9, 9, 4, 9, 7, 4, 7, 1, 5, 5,
        8, 1, 1, 2, 3, 

RuntimeError: ignored

In [6]:
xzz = {0:1, 1:2}